In [ ]:
! pip install -U -q sodapy confluent-kafka pandas 

wget https://dlcdn.apache.org/kafka/3.7.0/kafka_2.13-3.7.0.tgz
tar -xzf kafka_2.13-3.7.0.tgz
mv kafka_2.13-3.7.0  kafka
cd kafka

bin/zookeeper-server-start.sh config/zookeeper.properties
bin/kafka-server-start.sh config/server.properties

bin/kafka-topics.sh --create --topic service-requests-events --bootstrap-server localhost:9092
bin/kafka-console-consumer.sh --topic service-requests-events --from-beginning --bootstrap-server localhost:9092

In [1]:
from confluent_kafka import Producer
import pandas as pd
import json
from sodapy import Socrata
import time

In [2]:
data_url = "data.cityofnewyork.us"
data_set = "erm2-nwe9"
app_token = "tEeu8oBfckAQMIatiKafE7oeO"
bootstrap_servers = "localhost:9092"
topic = "service-requests-events"

In [3]:
client = Socrata(data_url, app_token)
producer = Producer({"bootstrap.servers": bootstrap_servers})
client.timeout = 30
data_offset = 20
api_limit = 20

In [13]:
def send_data_to_kafka():
    try:
        offset = api_limit + api_limit
        results = client.get(data_set, limit=api_limit, offset=offset)
        df = pd.DataFrame.from_records(results)

        for _, row in df.iterrows():
            producer.produce(topic, row.to_json())

        producer.flush()

        print("Data sent to Kafka topic successfully.")
    except Exception as ex:
        print(ex)

In [15]:
while True:
    send_data_to_kafka()
    time.sleep(5)

In [ ]:
# schedule.every(30).seconds.do(send_data_to_kafka)

In [ ]:
# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [4]:
# %%time
results = client.get(data_set, limit=10)
df = pd.DataFrame.from_records(results)
df.columns
# for index, row in df.iterrows():
#     producer.produce(topic, json.dumps(row.to_dict()).encode("utf-8"))

# producer.flush()

# print("Data sent to Kafka topic successfully.")

Index(['unique_key', 'created_date', 'agency', 'agency_name', 'complaint_type',
       'descriptor', 'location_type', 'incident_zip', 'incident_address',
       'street_name', 'address_type', 'city', 'facility_type', 'status',
       'due_date', 'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'latitude', 'longitude',
       'location', ':@computed_region_efsh_h5xi',
       ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q',
       ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih',
       ':@computed_region_7mpf_4k6g', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'landmark'],
      dtype='object')

In [7]:
df.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_7mpf_4k6g,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,landmark
0,60860295,2024-04-15T01:45:56.000,DSNY,Department of Sanitation,Graffiti,Graffiti,Residential,11211,216 DEVOE STREET,DEVOE STREET,...,36,2,30,56,56,NaN,NaN,NaN,NaN,NaN
1,60862716,2024-04-15T01:35:53.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11213,1389 ST JOHNS PLACE,ST JOHNS PLACE,...,16,2,49,49,49,SCHENECTADY AVENUE,UTICA AVENUE,SCHENECTADY AVENUE,UTICA AVENUE,ST JOHNS PLACE
2,60857082,2024-04-15T01:35:43.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10040,67 FORT GEORGE AVENUE,FORT GEORGE AVENUE,...,47,4,39,22,22,AMSTERDAM AVENUE,AUDUBON AVENUE,AMSTERDAM AVENUE,AUDUBON AVENUE,FORT GEORGE AVENUE
3,60862725,2024-04-15T01:35:42.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10468,165 WEST 197 STREET,WEST 197 STREET,...,48,5,29,33,33,WEBB AVENUE,SEDGWICK AVENUE,WEBB AVENUE,SEDGWICK AVENUE,WEST 197 STREET
4,60862729,2024-04-15T01:35:30.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11232,834 43 STREET,43 STREET,...,2,2,7,39,39,8 AVENUE,9 AVENUE,8 AVENUE,9 AVENUE,43 STREET


In [12]:
df1 = df.where(pd.notnull(df), None)
df1.to_json("data.json")

In [14]:
for _, row in df.iterrows():
  print(row.to_json())
  break

{'unique_key': '60860295', 'created_date': '2024-04-15T01:45:56.000', 'agency': 'DSNY', 'agency_name': 'Department of Sanitation', 'complaint_type': 'Graffiti', 'descriptor': 'Graffiti', 'location_type': 'Residential', 'incident_zip': '11211', 'incident_address': '216 DEVOE STREET', 'street_name': 'DEVOE STREET', 'address_type': 'ADDRESS', 'city': 'BROOKLYN', 'facility_type': 'N/A', 'status': 'Open', 'due_date': '2024-05-15T01:45:56.000', 'resolution_description': 'The graffiti on this property has been scheduled to be removed by the City.', 'resolution_action_updated_date': '2024-04-15T01:45:56.000', 'community_board': '01 BROOKLYN', 'bbl': '3027720010', 'borough': 'BROOKLYN', 'x_coordinate_state_plane': '1000260', 'y_coordinate_state_plane': '199399', 'open_data_channel_type': 'UNKNOWN', 'park_facility_name': 'Unspecified', 'park_borough': 'BROOKLYN', 'latitude': '40.71396645055422', 'longitude': '-73.94224860056295', 'location': {'latitude': '40.71396645055422', 'longitude': '-73.94